In [ ]:
import math
import numpy as np
#import h5py
#import matplotlib.pyplot as plt
import scipy
#from PIL import Image
from scipy import ndimage
import tensorflow as tf
from tensorflow.python.framework import ops
#from cnn_utils import *
## Load Libraries
#import os
#import requests, zipfile, io
#import tqdm
import pandas as pd
import numpy as np
import cv2


#%matplotlib inline
np.random.seed(1)

In [ ]:
# this is the current directory where files will get downloaded
#os.getcwd()

In [ ]:
# load data into platform
#url = requests.get('https://he-s3.s3.amazonaws.com/media/hackathon/deep-learning-challenge-1/identify-the-objects/a0409a00-8-dataset_dp.zip')
#data = zipfile.ZipFile(io.BytesIO(url.content))
#data.extractall()

In [ ]:
# check if the files have been download in current directory
#os.listdir()

In [ ]:
## load files
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.head()

In [ ]:
test.head()   #Used by hackerearth.com to evaluat submissions based on weighted F1 score

In [ ]:
# Before splitting , count the number of unique classes in training dataset
classes = train['label'].nunique()
print(classes)
classnames = pd.unique(train['label'])
print(classnames)

In [ ]:
# Im going to just randomly shuffle dataset and split it into train-test 4:1 ratio
df = pd.DataFrame(train)
#print(df)
train_random = df.sample(frac=0.8,random_state=200)
test_random = df.drop(train_random.index)
print(test_random.head())

In [ ]:
print(train_random['image_id'].values)

In [ ]:
# set path to read train and test image
TRAIN_PATH = 'train_img/'
TEST_PATH = 'test_img/'

In [ ]:
# function to read images as arrays
def read_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (64,64)) # you can resize to  (128,128) or (256,256)
    return img

In [ ]:
X_train_orig = []
X_test_orig = []

# looping over images in the directory
#for img in tqdm(train_random['image_id'].values):

for img in train_random['image_id'].values:
    X_train_orig.append(read_image(TRAIN_PATH+'{}.png'.format(img)))
for img in test_random['image_id'].values:
    X_test_orig.append(read_image(TRAIN_PATH+'{}.png'.format(img)))

In [ ]:
# Loading the data X
X_train_orig = np.array(X_train_orig)
X_test_orig = np.array(X_test_orig)
print(X_train_orig.shape)
print(X_test_orig.shape)

In [ ]:
train_labels = train_random['label'].values
test_labels = test_random['label'].values
print(train_labels.shape)
print(test_labels.shape)
print(train_labels)

In [ ]:
# convert class names from String to index values
for tstr in train_labels:
    train_labels[list(train_labels).index(tstr)] = list(classnames).index(tstr)
print(train_labels)
for tstr in test_labels:
    test_labels[list(test_labels).index(tstr)] = list(classnames).index(tstr)

In [ ]:
def convert_to_one_hot(data, countclass):
    Y = np.zeros([data.shape[0],countclass])
    print(Y.shape)
    #for notoeye in np.nditer(data ,["refs_ok"]):
    for notoeye in data:
        newrow = np.eye(data.shape[0])[notoeye-1]
        newrow = np.expand_dims(newrow, axis=1)
        np.concatenate((Y, newrow) , axis=1) 
    print(Y.shape)
    return Y

In [ ]:
train_labels = train_labels.reshape(-1)
print(train_labels.shape[0])
print(train_labels)

In [ ]:
# Loading the data Y by performing one hot encoding
print(train_labels.shape[0])
Y_train = convert_to_one_hot(train_labels, classes)

In [ ]:
Y_test = convert_to_one_hot(test_labels, classes)

In [ ]:
X_train = X_train_orig/255.
X_test = X_test_orig/255.
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))
conv_layers = {}

In [ ]:
# random_mini_batches
# minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)
def random_mini_batches(X, Y, minibatch_size = 64, seed = 0):
    m = X_train.shape[0]
    mini_batches = []
    np.random.seed(seed)
    
    # Shuffle (X, Y)
    permutation = np.random.permutation(m)
    X_shuffled = X[permutation,:,:,:]
    Y_shuffled = Y[permutation,:]

    #Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/minibatch_size)
    for k in range(0,num_complete_minibatches):
        mini_batch_x = X_shuffled[k*minibatch_size : k*minibatch_size+minibatch_size ,:,:,:]
        mini_batch_y = Y_shuffled[k*minibatch_size : k*minibatch_size+minibatch_size ,:]
        mini_batch = (mini_batch_x, mini_batch_y)
        mini_batches.append(mini_batch)
        
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % minibatch_size != 0:
        mini_batch_x = X_shuffled[num_complete_minibatches*minibatch_size : m ,:,:,:]
        mini_batch_y = Y_shuffled[num_complete_minibatches*minibatch_size : m ,:]
        mini_batch = (mini_batch_x, mini_batch_y)
        mini_batches.append(mini_batch)

    return mini_batches


In [ ]:
def create_placeholders(n_H0, n_W0, n_C0, n_y):
    """
    Creates the placeholders for the tensorflow session.
    
    Arguments:
    n_H0 -- scalar, height of an input image
    n_W0 -- scalar, width of an input image
    n_C0 -- scalar, number of channels of the input
    n_y -- scalar, number of classes
        
    Returns:
    X -- placeholder for the data input, of shape [None, n_H0, n_W0, n_C0] and dtype "float"
    Y -- placeholder for the input labels, of shape [None, n_y] and dtype "float"
    """

    ### START CODE HERE ### (≈2 lines)
    X = tf.placeholder(tf.float32 , shape = (None, n_H0, n_W0, n_C0) )
    Y = tf.placeholder(tf.float32 , shape = (None, n_y) )
    ### END CODE HERE ###
    
    return X, Y

In [ ]:
# GRADED FUNCTION: initialize_parameters

def initialize_parameters():
    """
    Initializes weight parameters to build a neural network with tensorflow. The shapes are:
                        W1 : [4, 4, 3, 8]
                        W2 : [2, 2, 8, 16]
    Returns:
    parameters -- a dictionary of tensors containing W1, W2
    """
    
    tf.set_random_seed(1)                              # so that your "random" numbers match ours
        
    ### START CODE HERE ### (approx. 2 lines of code)
    W1 = tf.get_variable("W1", [4,4,3,1], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    """
    W1 = tf.get_variable("W1", [4,4,3,96], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W2 = tf.get_variable("W2", [5,5,96,256], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W3 = tf.get_variable("W3", [3,3,256,384], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W4 = tf.get_variable("W4", [3,3,384,384], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W5 = tf.get_variable("W5", [3,3,384,256], initializer = tf.contrib.layers.xavier_initializer(seed = 0))"""
    ### END CODE HERE ###

   # parameters = {"W1": W1,
   #               "W2": W2,
   #               "W3": W3,
   #               "W4": W4,
   #               "W5": W5}
    
    parameters = {"W1": W1}
    
    return parameters

In [ ]:
# GRADED FUNCTION: forward_propagation

def forward_propagation(X, parameters):
    """
    Implements the forward propagation for the model:
    CONV2D -> RELU -> MAXPOOL -> CONV2D -> RELU -> MAXPOOL -> FLATTEN -> FULLYCONNECTED
    
    Arguments:
    X -- input dataset placeholder, of shape (input size, number of examples)
    parameters -- python dictionary containing your parameters "W1", "W2"
                  the shapes are given in initialize_parameters

    Returns:
    Z3 -- the output of the last LINEAR unit
    """
    
    # Retrieve the parameters from the dictionary "parameters" 
    W1 = parameters['W1']
    """W2 = parameters['W2']
    W3 = parameters['W3']
    W4 = parameters['W4']
    W5 = parameters['W5']"""
    
    ### START CODE HERE ###
    # CONV2D: stride of 1, padding 'VALID'
    Z1 = tf.nn.conv2d( X, W1, strides = [1,1,1,1], padding = 'VALID')
    # ?? have not taken b1 parameter
    # RELU
    A1 = tf.nn.relu(Z1)
    """
    # MAXPOOL: window 3x3, sride 2, padding 'VALID'
    P1 = tf.nn.max_pool(A1, ksize = [1,3,3,1], strides = [1,2,2,1], padding = 'VALID')
    # CONV2D: filters W2, stride 1, padding 'SAME'
    Z2 = tf.nn.conv2d(P1,W2, strides = [1,1,1,1], padding = 'SAME')
    # RELU
    A2 = tf.nn.relu(Z2)
    # MAXPOOL: window 2x2, stride 1, padding 'VALID'
    P2 = tf.nn.max_pool(A2, ksize = [1,2,2,1], strides = [1,1,1,1], padding = 'VALID')
    # CONV2D: filters W3, stride 1, padding 'SAME'
    Z3 = tf.nn.conv2d(P2,W3, strides = [1,1,1,1], padding = 'SAME')
    # RELU
    A3 = tf.nn.relu(Z3)
    # CONV2D: filters W4, stride 1, padding 'SAME'
    Z4 = tf.nn.conv2d(A3,W4, strides = [1,1,1,1], padding = 'SAME')
    # RELU
    A4 = tf.nn.relu(Z4)
    # CONV2D: filters W5, stride 1, padding 'SAME'
    Z5 = tf.nn.conv2d(A4,W5, strides = [1,1,1,1], padding = 'SAME')
    # RELU
    A5 = tf.nn.relu(Z5)
    # MAXPOOL: window 3x3, stride 2, padding 'VALID'
    P3 = tf.nn.max_pool(A5, ksize = [1,3,3,1], strides = [1,2,2,1], padding = 'VALID')
    # FLATTEN
    P3 = tf.contrib.layers.flatten(P3)
    # FULLY-CONNECTED with non-linear activation function.
    A6 = tf.contrib.layers.fully_connected(P3, 4096)
    A7 = tf.contrib.layers.fully_connected(A6, 4096)    
    # FULLY-CONNECTED without non-linear activation function (not not call softmax).
    # 6 neurons in output layer. Hint: one of the arguments should be "activation_fn=None" 
    Z8 = tf.contrib.layers.fully_connected(A7, 6, activation_fn=None)
    """
    # FLATTEN
    P1 = tf.contrib.layers.flatten(A1)
    Z8 = tf.contrib.layers.fully_connected(P1, 25, activation_fn=None)
    ### END CODE HERE ###

    return Z8

In [ ]:
# GRADED FUNCTION: compute_cost 

def compute_cost(Z8, Y):
    """
    Computes the cost
    
    Arguments:
    Z8 -- output of forward propagation (output of the last LINEAR unit), of shape (25, number of examples)
    Y -- "true" labels vector placeholder, same shape as Z8
    
    Returns:
    cost - Tensor of the cost function
    """
    
    ### START CODE HERE ### (1 line of code)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits = Z8, labels = Y) )
    ### END CODE HERE ###
    
    return cost

In [ ]:
# GRADED FUNCTION: model

def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.009,
          num_epochs = 100, minibatch_size = 64, print_cost = True):
    """
    Implements a three-layer ConvNet in Tensorflow:
    CONV2D -> RELU -> MAXPOOL -> CONV2D -> RELU -> MAXPOOL -> FLATTEN -> FULLYCONNECTED
    
    Arguments:
    X_train -- training set, of shape (None, 64, 64, 3)
    Y_train -- test set, of shape (None, n_y = 6)
    X_test -- training set, of shape (None, 64, 64, 3)
    Y_test -- test set, of shape (None, n_y = 6)
    learning_rate -- learning rate of the optimization
    num_epochs -- number of epochs of the optimization loop
    minibatch_size -- size of a minibatch
    print_cost -- True to print the cost every 100 epochs
    
    Returns:
    train_accuracy -- real number, accuracy on the train set (X_train)
    test_accuracy -- real number, testing accuracy on the test set (X_test)
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
    tf.set_random_seed(1)                             # to keep results consistent (tensorflow seed)
    seed = 3                                          # to keep results consistent (numpy seed)
    (m, n_H0, n_W0, n_C0) = X_train.shape             
    n_y = Y_train.shape[1]                            
    costs = []                                        # To keep track of the cost
    
    # Create Placeholders of the correct shape
    ### START CODE HERE ### (1 line)
    X, Y = create_placeholders(n_H0, n_W0, n_C0, n_y)
    ### END CODE HERE ###

    # Initialize parameters
    ### START CODE HERE ### (1 line)
    parameters = initialize_parameters()
    ### END CODE HERE ###
    
    # Forward propagation: Build the forward propagation in the tensorflow graph
    ### START CODE HERE ### (1 line)
    Z8 = forward_propagation(X, parameters)
    ### END CODE HERE ###
    
    # Cost function: Add cost function to tensorflow graph
    ### START CODE HERE ### (1 line)
    cost = compute_cost(Z8, Y)
    ### END CODE HERE ###
    
    # Backpropagation: Define the tensorflow optimizer. Use an AdamOptimizer that minimizes the cost.
    ### START CODE HERE ### (1 line)
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
    ### END CODE HERE ###
    
    # Initialize all the variables globally
    init = tf.global_variables_initializer()
     
    # Start the session to compute the tensorflow graph
    with tf.Session() as sess:
        
        # Run the initialization
        sess.run(init)
        
        # Do the training loop
        for epoch in range(num_epochs):

            minibatch_cost = 0.
            num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
            seed = seed + 1
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)

            for minibatch in minibatches:

                # Select a minibatch
                (minibatch_X, minibatch_Y) = minibatch
                # IMPORTANT: The line that runs the graph on a minibatch.
                # Run the session to execute the optimizer and the cost, the feedict should contain a minibatch for (X,Y).
                ### START CODE HERE ### (1 line)
                _ , temp_cost = sess.run([optimizer, cost], feed_dict = {X:minibatch_X , Y:minibatch_Y})
                ### END CODE HERE ###
                
                minibatch_cost += temp_cost / num_minibatches
                

            # Print the cost every epoch
            if print_cost == True and epoch % 5 == 0:
                print ("Cost after epoch %i: %f" % (epoch, minibatch_cost))
            if print_cost == True and epoch % 1 == 0:
                costs.append(minibatch_cost)
        
        
        # plot the cost
        #plt.plot(np.squeeze(costs))
        #plt.ylabel('cost')
        #plt.xlabel('iterations (per tens)')
        #plt.title("Learning rate =" + str(learning_rate))
        #plt.show()

        # Calculate the correct predictions
        predict_op = tf.argmax(Z8, 1)
        correct_prediction = tf.equal(predict_op, tf.argmax(Y, 1))
        
        # Calculate accuracy on the test set
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        print(accuracy)
        train_accuracy = accuracy.eval({X: X_train, Y: Y_train})
        test_accuracy = accuracy.eval({X: X_test, Y: Y_test})
        print("Train Accuracy:", train_accuracy)
        print("Test Accuracy:", test_accuracy)
                
        return train_accuracy, test_accuracy, parameters

In [ ]:
_, _, parameters = model(X_train, Y_train, X_test, Y_test, num_epochs = 5)